In [18]:
from dotenv import load_dotenv
load_dotenv()

True

In [1]:
from langgraph.types import Command,Literal
from langchain_core.messages import HumanMessage,AIMessage,BaseMessage
from langgraph.graph import MessagesState,START,END,StateGraph
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_experimental.tools import PythonREPLTool
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from typing import Annotated
from pprint import pprint

c:\Users\gokul\anaconda3\envs\langgraphenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def make_system_prompt(suffix: str) -> str:
    return (
        "You are a helpful AI assistant, collaborating with other assistants."
        " Use the provided tools to progress towards answering the question."
        " If you are unable to fully answer, that's OK, another assistant with different tools "
        " will help where you left off. Execute what you can to make progress."
        " If you or any of the other assistants have the final answer or deliverable,"
        " prefix your response with FINAL ANSWER so the team knows to stop."
        f"\n{suffix}"
    )

In [3]:
repl = PythonREPLTool()

In [4]:
@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    result_str = f"Successfully executed:\n\`\`\`python\n{code}\n\`\`\`\nStdout: {result}"
    return (
        result_str + "\n\nIf you have completed all tasks, respond with FINAL ANSWER."
    )

In [5]:
model = ChatGoogleGenerativeAI(model='gemini-2.0-flash',temperature=0.1)

In [6]:
agent = create_react_agent(model=model,tools=[python_repl_tool],prompt="You are an agent which can use python tool to generate answers . Always return executed results")

In [7]:
# agent.invoke({'messages':"Hello there!"})

In [8]:
# response = agent.invoke({'messages':"Can you write a python code for 15th fibonacci series number? execute it and give me the output"})

In [9]:
# for msg in response['messages']:
#     msg.pretty_print()

In [10]:
def code_agent(state:MessagesState):
    msg_prompt = state['messages'][-1].content
    response = agent.invoke({'messages':HumanMessage(content=msg_prompt)})
    state['messages']=state['messages']+[response['messages'][-1]]

    return state

In [11]:
graph = StateGraph(MessagesState)

In [12]:
graph.add_node("agent",code_agent)

In [13]:
graph.add_edge(START,"agent")
graph.add_edge("agent",END)

In [14]:
graph_built = graph.compile()

In [15]:
# graph.compile()

In [16]:
response= graph_built.invoke({'messages':HumanMessage(content='hello there!')})
for msg in response['messages']:
    msg.pretty_print()

================================ Human Message =================================

hello there!
================================== Ai Message ==================================

Hello! How can I help you today?


In [ ]:
events = graph_built.stream(
    {
        "messages": HumanMessage(content='Hello there!')
    },
    {"recursion_limit": 150},
)

for event in events:
    # Access the messages in each event
    for message in event['agent']['messages']:
        message.pretty_print()  # This uses the built-in pretty_print method of message objects


================================ Human Message =================================

Hello there!
--------------------------------------------------
================================== Ai Message ==================================

Hi, how can I help you today?
--------------------------------------------------
